## Do all preliminary operations

In [ ]:
%matplotlib inline

In [ ]:
import os
import json
import matplotlib.pyplot as plt

In [ ]:
json_wd = os.path.join(os.getcwd(),'json')
print (json_wd)

In [ ]:
from os import listdir
from os.path import isfile, join

jsonfiles = [f for f in listdir(json_wd) if isfile(join(json_wd, f))]

In [ ]:
print(len(jsonfiles))

## Import JSON files

In [ ]:
#input all json files and load into list
all_files=[]
for filename in jsonfiles:
    print(filename)#allows for easy debugging of JSON files
    file=open(os.path.join(json_wd, filename))
    data=json.load(file)
    all_files.append(data)
    file.close()
    #data=json.loads(filename)
print (all_files[0])

## Make all the links using tags

In [ ]:
all_links=[]
dataset_list=[]
for index, file in enumerate(all_files):
    #print (list(file.keys())[0], "is the name of the dataset")
    name=list(file.keys())[0]
    dataset_list.append(name)
    #print (list(file[name].keys()), "are the column names")
    for col in file[name].keys():
        #print ("Under column ", col, "are tags ", file[name][col])
        for tag in file[name][col]:
            #print ("Tag: ", tag)
            for secondindex in range (index+1, len(all_files)):
                secondname=list(all_files[secondindex].keys())[0]
                for secondcol in all_files[secondindex][secondname].keys():
                    #print ("Under second column ", secondcol, "are tags ", all_files[secondindex][secondname][secondcol])
                    for secondtag in all_files[secondindex][secondname][secondcol]:
                        #print ("Tag: ", secondtag)
                        #do all checking for common tags here
                        if tag.upper()==secondtag.upper() and index is not secondindex:
                            print (name, "and", secondname, "are linked at column", col, "and column", secondcol, "through tag", tag)
                            all_links.append([name.upper(), col.upper(), tag.upper() , secondname.upper(), secondcol.upper(), secondtag.upper()])
print (len(all_links))

## Reformat list for easy access

In [ ]:
full_tree=[]#make tag most important
for sublist in all_links:
    tag=sublist[2]
    col1=sublist[1]
    col2=sublist[4]
    dataset1=sublist[0]
    dataset2=sublist[3]
    full_tree.append([tag, col1, dataset1, col2, dataset2])
#print (full_tree)

# Basic Data analysis 

## Number of times each tag is linked

In [ ]:
tag_dict={}
for sublist in full_tree:
    try:
        tag_dict[sublist[0]]+=1
    except Exception:
        tag_dict[sublist[0]]=1
print (tag_dict)

In [ ]:
li=tag_dict.keys()
new_li=[]
for l in li:
    new_li.append(l[0:3])
plt.bar(new_li, tag_dict.values(), .5, color='b')#resize as needed

## Number of times each column has associated tags

In [ ]:
col_dict={}
for sublist in full_tree:
    try:
        col_dict[sublist[1]]+=1
    except Exception:
        col_dict[sublist[1]]=1
    try:
        col_dict[sublist[3]]+=1
    except Exception:
        col_dict[sublist[3]]=1
print (col_dict)

In [ ]:
li=col_dict.keys()
new_li=[]
for l in li:
    new_li.append(l[0:3])
plt.bar(new_li, col_dict.values(), .5, color='g')

## Number of links each dataset has

In [ ]:
dataset_dict={}
for sublist in full_tree:
    try:
        dataset_dict[sublist[2]]+=1
    except Exception:
        dataset_dict[sublist[2]]=1
    try:
        dataset_dict[sublist[4]]+=1
    except Exception:
        dataset_dict[sublist[4]]=1
print (dataset_dict)

In [ ]:
li=dataset_dict.keys()
new_li=[]
for l in li:
    new_li.append(l[0:3])
plt.bar(new_li, dataset_dict.values(), .5, color='r')

## Organize linking by tree

In [ ]:
full_tree=sorted(full_tree)

In [ ]:
from anytree import Node, RenderTree, Resolver
from anytree.search import find
resolver=Resolver('name')
dummy=Node('All_Tags')
for sublist in full_tree:#Tags
    f=find(dummy, lambda node: node.name == sublist[0] and node.depth== 1)
    if f is None:
        n=Node(sublist[0], parent=dummy)
        l=[]
        for lis in full_tree:
            if lis[1] not in l and lis[0] == n.name:
                l.append(lis[1])
            if lis[3] not in l and lis[0] == n.name:
                l.append(lis[3])
        for lis in l:
            p=Node(lis, parent=n)
            q=[]
            for li in full_tree:
                #print (li[2],"second:" ,li[4])
                #print ("next")
                if li[0] == n.name and li[1]==p.name:
                    q.append(li[2])
                if li[0] == n.name  and li[3]==p.name:
                    q.append(li[4])
            q=list(set(q))
            for li in q:
                r=Node(li, parent=p)
            
#for pre, fill, node in RenderTree(dummy):
#    print("%s%s" % (pre, node.name))

In [ ]:
# from anytree import Node, RenderTree, Resolver
# from anytree.search import find
# resolver=Resolver('name')
# dummy=Node('All_Tags')
# for sublist in full_tree:#Tags
#     f=find(dummy, lambda node: node.name == sublist[0])
#     if f is None:
#         n=Node(sublist[0], parent=dummy)
# for sublist in full_tree:#Columns
#     f=find(dummy, lambda node: node.name == sublist[1])
#     if f is None:
#         g=find(dummy, lambda node: node.name == sublist[0])
#         n=Node(sublist[1], parent=g)   
#     f=find(dummy, lambda node: node.name == sublist[3])
#     if f is None:
#         g=find(dummy, lambda node: node.name == sublist[0])
#         n=Node(sublist[3], parent=g) 
# o=None
# p=None
# for sublist in full_tree:#todo fix this part up
#     try:
#         g=find(dummy, lambda node: node.name == sublist[1])
#         n=Node(sublist[2], parent=g)
#     except Exception:
#         pass
#     try:
#         g=find(dummy, lambda node: node.name == sublist[3])
#         n=Node(sublist[4], parent=g)
#     except Exception:
#         pass

In [ ]:
dummy.children #all the tags should be listed

In [ ]:
# all_leafs=[]
# should_leafs=[]
# def get_leafs(node):
#     for child in node.children:
#         get_leafs(child)
#     if len(node.children) is 0 and node.depth is 3:
#         all_leafs.append(node)
# def should_be_leafs(node):
#     for child in node.children:
#         should_be_leafs(child)
#     if len(node.children) is 0:
#         if node.name not in should_leafs  and node.depth is 3:
#             should_leafs.append(node.name)

In [ ]:
# get_leafs(dummy)
# should_be_leafs(dummy)
#print (all_leafs)

In [ ]:
for pre, fill, node in RenderTree(dummy):
    print("%s%s" % (pre, node.name))

In [ ]:
#This will allow working with 1 subtree by tag at a time
l=dummy.children
print (l[0])
for pre, fill, node in RenderTree(l[0]):
    print("%s%s" % (pre, node.name))

In [ ]:
from anytree.exporter import DotExporter
DotExporter(l[0]).to_dotfile("tree.dot")

In [ ]:
#for some reason this does not work. Just use regular command prompt
#import subprocess
#subprocess.Popen('dot tree.dot -T png -o tree.png')